In [ ]:
cd drive/MyDrive/LLaVA

/content/drive/MyDrive/LLaVA


In [ ]:
!pip install --upgrade pip
!git pull
!pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Already up to date.
Obtaining file:///content/drive/MyDrive/LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model, load_images

import argparse
import torch
import random
import pickle as pkl

from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

import pandas as pd
import gc
import numpy as np

import requests
from PIL import Image
from io import BytesIO
import json
import re
import os
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# model_path = "liuhaotian/llava-v1.6-vicuna-7b"
# model_path = "liuhaotian/llava-v1.5-13b"
model_path = "liuhaotian/llava-v1-0719-336px-lora-merge-vicuna-13b-v1.3"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    load_4bit=True,
    model_name=get_model_name_from_path(model_path)
)

/content/drive/MyDrive/LLaVA/llava/model/builder.py:51: UserWarning: There is `lora` in model name but no `model_base` is provided. If you are loading a LoRA model, please provide the `model_base` argument. Detailed instruction: https://github.com/haotian-liu/LLaVA#launch-a-model-worker-lora-weights-unmerged.
  warnings.warn('There is `lora` in model name but no `model_base` is provided. If you are loading a LoRA model, please provide the `model_base` argument. Detailed instruction: https://github.com/haotian-liu/LLaVA#launch-a-model-worker-lora-weights-unmerged.')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authenticati

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at liuhaotian/llava-v1-0719-336px-lora-merge-vicuna-13b-v1.3 were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_ids', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.laye

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
def eval(tokenizer,
         model,
         image_processor,
         model_path,
         qs,
         image_files,
         temperature=1,
         top_p=0.8,
         num_beams=5,
         max_new_tokens=17):

  disable_torch_init()

  model_name = get_model_name_from_path(model_path)
  image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN

  if IMAGE_PLACEHOLDER in qs:
    if model.config.mm_use_im_start_end:
        qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
    else:
        qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
  else:
      if model.config.mm_use_im_start_end:
          qs = image_token_se + "\n" + qs
      else:
          qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

  if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
  elif "mistral" in model_name.lower():
      conv_mode = "mistral_instruct"
  elif "v1.6-34b" in model_name.lower():
      conv_mode = "chatml_direct"
  elif "v1" in model_name.lower():
      conv_mode = "llava_v1"
  elif "mpt" in model_name.lower():
      conv_mode = "mpt"
  else:
      conv_mode = "llava_v0"

  conv = conv_templates[conv_mode].copy()
  conv.append_message(conv.roles[0], qs)
  conv.append_message(conv.roles[1], None)

  prompt = conv.get_prompt()

  images = load_images(image_files)
  image_sizes = [x.size for x in images]
  images_tensor = process_images(
      images,
      image_processor,
      model.config
  ).to(model.device, dtype=torch.float16)

  input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .cuda()
  )

  with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=images_tensor,
        image_sizes=image_sizes,
        do_sample=True if temperature > 0 else False,
        temperature=temperature,
        top_p=top_p,
        num_beams=num_beams,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return outputs

In [ ]:
def preprocess(df, name_to_value):
  id_to_prompt = {}
  id_to_label = {}

  # df['prompt'] = df.apply(lambda row: f"Question: What is {row['attribute_names']} of this product?\nContext: [Category] {row['category']} {row['texts']}.\nYou must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  # df['prompt'] = df.apply(lambda row: f"Question: What is {row['attribute_names']} of this product?\nContext: [Category] {row['category']} {row['texts']}.\nYou must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  # df['prompt'] = df.apply(lambda row: f"What is {row['attribute_names']} of this product?[Category] {row['category']} {row['texts']}.Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  # df['prompt'] = df.apply(lambda row: f"[Category] {row['category']} {row['texts']}. What is {row['attribute_names']} of this product? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.", axis=1)
  # df['prompt'] = df.apply(lambda row: f"[Category] {row['category']} {row['texts']}. What is {row['attribute_names']} of this product based on the given information and the given image? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.", axis=1)
  # df['prompt'] = df.apply(lambda row: f"[Category] {row['category']} {row['texts']}. Which one of {name_to_value[row['attribute_names']]} is the {row['attribute_names']} of this product? Answer with the option from the given choices directly.", axis=1)
  # df['prompt'] = df.apply(lambda row: f"{row['texts']}. What is the {row['attribute_names']} of this product? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.", axis=1)
  # df['prompt'] = df.apply(lambda row: f"{row['texts']}. Based on the description and the image, what is the {row['attribute_names']} of this product? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.", axis=1)
  # df['prompt'] = df.apply(lambda row: f"What is the {row['attribute_names']} of this product: {row['texts']}? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.", axis=1)



  #only v1
  # df['prompt'] = df.apply(lambda row: f"Question: What is {row['attribute_names']} of this product?\nContext: [Category] {row['category']} {row['texts']}.\nYou must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  # df['prompt'] = df.apply(lambda row: f"Question: What is {row['attribute_names']} of this product?\nContext: [Category] {row['category']} {row['texts']}.\nYou must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  df['prompt'] = df.apply(lambda row: f"Question: What is {row['attribute_names']} of this product?{row['category']} {row['texts']}.You must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}. \nAnswer:", axis=1)
  # df['prompt'] = df.apply(lambda row: f"What is {row['attribute_names']} of this product?{row['category']} {row['texts']}.You must only answer the question with exactly one of the following options {name_to_value[row['attribute_names']]}.", axis=1)

  for row in df.itertuples():
    id_to_prompt[row.id] = row.prompt
    id_to_label[row.id] = row.attribute_values

  return id_to_prompt, id_to_label


In [ ]:
file_names = [
    'Clothing_annotated_final.tsv',
    'Jewlery_and_General_Apparel_annotated_final.tsv',
    'Footwear_annotated_final.tsv',
    'Food_annotated_final.tsv',
    'Home_annotated_final.tsv'
]

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
def inference(model, model_path, tokenizer, image_processor, id_to_prompts, id_to_labels, img_dir, temperature=0.9, top_p=0.9):
  id_to_pred = {}
  preds = []
  labels = []

  for id in id_to_prompts:
    labels.append(id_to_labels[id])
    text = id_to_prompts[id]
    img_name = f'{id}.jpg'
    img_path = os.path.join(img_dir, img_name)

    if not os.path.exists(img_path):
      continue

    output = eval(tokenizer,
                  model,
                  image_processor,
                  model_path,
                  text,
                  [img_path],
                  temperature=temperature,
                  top_p=top_p,
                  num_beams=5,
                  max_new_tokens=17)[0]
    preds.append(output)
    id_to_pred[id] = output

    torch.cuda.empty_cache()
    gc.collect()

  return id_to_pred, preds, labels


In [ ]:
def calculate_metrics(predictions, targets):
  binary_predictions = [1 if target.lower() in prediction.lower() else 0 for target, prediction in zip(targets, predictions)]
  binary_targets = [1] * len(targets)

  accuracy = sum(binary_predictions)
  micro_f1 = f1_score(binary_targets, binary_predictions, average='micro')
  return accuracy, micro_f1

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)  # You can use any number as the seed

In [ ]:
img_dir = "../AVE/datasets/image_final"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, model_path, tokenizer, image_processor, id_to_prompts, id_to_labels, img_dir, temperature=0.9, top_p=0.9)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)



  with open(f'../AVE/datasets/results/llava_v1_13b_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

For Clothing_annotated_final multi modalities micro_f1 was 0.22123893805309736
For Jewlery_and_General_Apparel_annotated_final multi modalities micro_f1 was 0.6272727272727273
For Footwear_annotated_final multi modalities micro_f1 was 0.39747634069400634
For Food_annotated_final multi modalities micro_f1 was 0.49230769230769234
For Home_annotated_final multi modalities micro_f1 was 0.5776805251641138


In [ ]:
img_dir = "../AVE/datasets/image_final"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, model_path, tokenizer, image_processor, id_to_prompts, id_to_labels, img_dir, temperature=0.9, top_p=0.9)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)



  # with open(f'../AVE/datasets/results/llava_v1_13b_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
  #     pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

For Clothing_annotated_final multi modalities micro_f1 was 0.22123893805309736


In [ ]:
# 0.190265
# 0.19911504424778761
# 0.23893805309734514
# 0.22123893805309736

In [ ]:
del tokenizer
del model
del image_processor
del context_len

In [ ]:
model_path = "liuhaotian/llava-v1.6-vicuna-7b"
# # model_path = "liuhaotian/llava-v1.5-13b"
# model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    load_4bit=True,
    model_name=get_model_name_from_path(model_path)
)

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/py

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

In [ ]:
img_dir = "../AVE/datasets/image_final"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, model_path, tokenizer, image_processor, id_to_prompts, id_to_labels, img_dir, temperature=0.9, top_p=0.9)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)



  with open(f'../AVE/datasets/results/llava_1.6_7b_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

For Clothing_annotated_final multi modalities micro_f1 was 0.252212389380531
For Jewlery_and_General_Apparel_annotated_final multi modalities micro_f1 was 0.7772727272727271
For Footwear_annotated_final multi modalities micro_f1 was 0.3659305993690851
For Food_annotated_final multi modalities micro_f1 was 0.7666666666666667
For Home_annotated_final multi modalities micro_f1 was 0.6805251641137856


In [ ]:
del tokenizer
del model
del image_processor
del context_len

In [ ]:
model_path = "liuhaotian/llava-llama-2-7b-chat-lightning-lora-preview"
# # model_path = "liuhaotian/llava-v1.5-13b"
# model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    load_4bit=True,
    model_name=get_model_name_from_path(model_path)
)

/content/drive/MyDrive/LLaVA/llava/model/builder.py:51: UserWarning: There is `lora` in model name but no `model_base` is provided. If you are loading a LoRA model, please provide the `model_base` argument. Detailed instruction: https://github.com/haotian-liu/LLaVA#launch-a-model-worker-lora-weights-unmerged.
  warnings.warn('There is `lora` in model name but no `model_base` is provided. If you are loading a LoRA model, please provide the `model_base` argument. Detailed instruction: https://github.com/haotian-liu/LLaVA#launch-a-model-worker-lora-weights-unmerged.')


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

OSError: Can't load tokenizer for 'liuhaotian/llava-llama-2-7b-chat-lightning-lora-preview'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'liuhaotian/llava-llama-2-7b-chat-lightning-lora-preview' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [ ]:
img_dir = "../AVE/datasets/image_final"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, model_path, tokenizer, image_processor, id_to_prompts, id_to_labels, img_dir, temperature=0.9, top_p=0.9)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)



  with open(f'../AVE/datasets/results/llava_v1_7b_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
img_dir = "../AVE/datasets/image_final"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  for temp in [0.9, 0.7, 0.5, 0.3, 0.1, 0]:
    for tp in [0.5, 0.6, 0.7, 0.8, 0.9]:
      id_to_pred, preds, labels = inference(id_to_prompts, id_to_labels, img_dir, temperature=temp, top_p=tp)
      both_acc, both_micro_f1 = calculate_metrics(preds, labels)



      # with open(f'../AVE/datasets/results/llava_1.5_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      #     pkl.dump(id_to_pred, f)

      print(f'For {dataset[:-4]} with temperature {temp} and top_p {tp} the micro_f1 was {both_micro_f1}')

      torch.cuda.empty_cache()
      gc.collect()

For Clothing_annotated_final with temperature 0.9 and top_p 0.5 the micro_f1 was 0.48672566371681414
For Clothing_annotated_final with temperature 0.9 and top_p 0.6 the micro_f1 was 0.4823008849557522
For Clothing_annotated_final with temperature 0.9 and top_p 0.7 the micro_f1 was 0.47345132743362833
For Clothing_annotated_final with temperature 0.9 and top_p 0.8 the micro_f1 was 0.4690265486725664
For Clothing_annotated_final with temperature 0.9 and top_p 0.9 the micro_f1 was 0.4911504424778761
For Clothing_annotated_final with temperature 0.7 and top_p 0.5 the micro_f1 was 0.48672566371681414
For Clothing_annotated_final with temperature 0.7 and top_p 0.6 the micro_f1 was 0.48672566371681414
For Clothing_annotated_final with temperature 0.7 and top_p 0.7 the micro_f1 was 0.48672566371681414
For Clothing_annotated_final with temperature 0.7 and top_p 0.8 the micro_f1 was 0.4778761061946903
For Clothing_annotated_final with temperature 0.7 and top_p 0.9 the micro_f1 was 0.486725663716

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


For Clothing_annotated_final with temperature 0 and top_p 0.5 the micro_f1 was 0.45132743362831856


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


For Clothing_annotated_final with temperature 0 and top_p 0.6 the micro_f1 was 0.45132743362831856


...

For Clothing_annotated_final with temperature 0 and top_p 0.7 the micro_f1 was 0.45132743362831856


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


For Clothing_annotated_final with temperature 0 and top_p 0.8 the micro_f1 was 0.45132743362831856


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


For Clothing_annotated_final with temperature 0 and top_p 0.9 the micro_f1 was 0.45132743362831856
